In [1]:
import pandas as pd

In [2]:
df_Xtrain = pd.read_csv("Data/x_train_update.csv")

In [3]:
df_Xtrain.head()

,Unnamed: 0,designation,description,productid,imageid
0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237
2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496
4,4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


In [4]:
df_Xtrain.shape

(84916, 5)

In [5]:
df_Xtrain.isna().sum()

Unnamed: 0         0
designation        0
description    29800
productid          0
imageid            0
dtype: int64

In [6]:
df_Xtest = pd.read_csv("Data/x_test_update.csv")

In [7]:
df_Xtest.head()

,Unnamed: 0,designation,description,productid,imageid
0,84916,Folkmanis Puppets - 2732 - Marionnette Et Théâ...,NaN,516376098,1019294171
1,84917,Porte Flamme Gaxix - Flamebringer Gaxix - 136/...,NaN,133389013,1274228667
2,84918,Pompe de filtration Speck Badu 95,NaN,4128438366,1295960357
3,84919,Robot de piscine électrique,<p>Ce robot de piscine d&#39;un design innovan...,3929899732,1265224052
4,84920,Hsm Destructeur Securio C16 Coupe Crois¿E: 4 X...,NaN,152993898,940543690


In [8]:
df_Xtest.shape

(13812, 5)

In [9]:
df_Xtest.isna().sum()

Unnamed: 0        0
designation       0
description    4886
productid         0
imageid           0
dtype: int64

In [10]:
df_Ytrain = pd.read_csv("Data/Y_test.csv")

In [11]:
df_Ytest = pd.read_csv("Data/Y_train.csv")

In [12]:
df_Ytrain.shape

(13812, 2)

In [13]:
df_Ytrain.head()

,Unnamed: 0,prdtypecode
0,84916,10
1,84917,10
2,84918,10
3,84919,10
4,84920,10


In [14]:
df_Ytrain.isna().sum()

Unnamed: 0     0
prdtypecode    0
dtype: int64

In [15]:
df_Ytest.shape

(84916, 2)

In [16]:
df_Ytest.head()

,Unnamed: 0,prdtypecode
0,0,10
1,1,2280
2,2,50
3,3,1280
4,4,2705


In [17]:
df_Ytest.isna().sum()

Unnamed: 0     0
prdtypecode    0
dtype: int64

In [18]:
#df_Xtrain_image_sample1 = pd.read_csv("Data/images/image_train/image_528113_product_923222.jpg")

In [19]:
df_Xtrain.columns

Index(['Unnamed: 0', 'designation', 'description', 'productid', 'imageid'], dtype='object')

In [20]:
df_Xtest.columns

Index(['Unnamed: 0', 'designation', 'description', 'productid', 'imageid'], dtype='object')

In [21]:
df_Ytrain.columns

Index(['Unnamed: 0', 'prdtypecode'], dtype='object')

In [22]:
df_Ytest.columns

Index(['Unnamed: 0', 'prdtypecode'], dtype='object')

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# ✅ Chargement des données
print("Chargement des données...")
df_Xtrain = pd.read_csv("Data/X_train_update.csv")
df_Xtest = pd.read_csv("Data/X_test_update.csv")
df_Ytrain = pd.read_csv("Data/Y_train.csv")
df_Ytest = pd.read_csv("Data/Y_test.csv")
print("Données chargées avec succès !")

# ✅ Correction du warning en évitant inplace=True
print("Traitement des valeurs manquantes...")
df_Xtrain = df_Xtrain.copy()
df_Xtest = df_Xtest.copy()

df_Xtrain['description'] = df_Xtrain['description'].fillna("aucune description")
df_Xtest['description'] = df_Xtest['description'].fillna("aucune description")
print("Valeurs manquantes traitées !")

# ✅ Fusion des features avec les labels
print("Fusion des datasets...")
df_train = df_Xtrain.merge(df_Ytrain, on="Unnamed: 0").drop(columns=["Unnamed: 0"])
df_test = df_Xtest.merge(df_Ytest, on="Unnamed: 0").drop(columns=["Unnamed: 0"])
print("Fusion terminée !")

# ✅ Création des variables X (features) et y (labels)
X_train_text = df_train["description"]
X_test_text = df_test["description"]
y_train = df_train["prdtypecode"]
y_test = df_test["prdtypecode"]

# ✅ Création d'un pipeline avec un vectoriseur TF-IDF et un modèle de classification
print("Création du pipeline et début de l'entraînement...")
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words=None, max_features=10000)),  # Vectorisation des descriptions
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))  # Modèle
])

#from nltk.corpus import stopwords
#stop_words = stopwords.words('french')
#vectorizer = TfidfVectorizer(stop_words=stop_words)

# ✅ Mesure du temps d'entraînement
start_time = time.time()
pipeline.fit(X_train_text, y_train)
training_time = time.time() - start_time
print(f"Modèle entraîné en {training_time:.2f} secondes.")

# ✅ Prédictions et évaluation
print("Prédiction et évaluation du modèle...")
y_pred = pipeline.predict(X_test_text)
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {accuracy:.4f}")

print("Fin du processus ! 🎯")


Chargement des données...
Données chargées avec succès !
Traitement des valeurs manquantes...
Valeurs manquantes traitées !
Fusion des datasets...
Fusion terminée !
Création du pipeline et début de l'entraînement...


In [ ]:
pip install torch torchvision transformers Pillow

In [ ]:
# Traintement des données textuels et visuels 

In [ ]:
pip install --upgrade optree

In [ ]:
pip install transformers torch

In [2]:
import torch
from transformers import BlipProcessor, BlipModel, BertTokenizer, BertModel
import pandas as pd
import os
from PIL import Image
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# 1. Chargement des données
print("Chargement des données...")
df_Xtrain = pd.read_csv("Data/X_train_update.csv")
df_Xtest = pd.read_csv("Data/X_test_update.csv")
df_Ytrain = pd.read_csv("Data/Y_train.csv")
df_Ytest = pd.read_csv("Data/Y_test.csv")
print("Données chargées avec succès !")

# 2. Traitement des valeurs manquantes
print("Traitement des valeurs manquantes...")
df_Xtrain.fillna(0, inplace=True)
df_Xtest.fillna(0, inplace=True)
df_Ytrain.fillna(0, inplace=True)
df_Ytest.fillna(0, inplace=True)
print("Valeurs manquantes traitées !")

# 3. Fusion des datasets
print("Fusion des datasets...")
df_train = pd.concat([df_Xtrain, df_Ytrain], axis=1)
df_test = pd.concat([df_Xtest, df_Ytest], axis=1)
print("Fusion terminée !")

# 4. Chargement du modèle et du processeur BLIP pour l'extraction d'images
print("Chargement du modèle BLIP...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipModel.from_pretrained("Salesforce/blip-image-captioning-base")
model.eval()  # Mode évaluation pour éviter la mise à jour des poids
print("Modèle BLIP chargé avec succès !")

# 5. Fonction pour extraire les caractéristiques d'une image
def extract_image_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        features = model.visual_encoder(**inputs)

    return features.squeeze().numpy()

# 6. Extraire les caractéristiques des images d'entraînement
image_train_folder = "Data/images/image_train"
image_train_features = []

print("Extraction des caractéristiques des images d'entraînement...")
for image_file in os.listdir(image_train_folder):
    image_path = os.path.join(image_train_folder, image_file)
    image_features = extract_image_features(image_path)
    image_train_features.append(image_features)
print(f"Caractéristiques des images d'entraînement extraites : {len(image_train_features)} images.")

# 7. Extraire les caractéristiques des images de test
image_test_folder = "Data/images/image_test"
image_test_features = []

print("Extraction des caractéristiques des images de test...")
for image_file in os.listdir(image_test_folder):
    image_path = os.path.join(image_test_folder, image_file)
    image_features = extract_image_features(image_path)
    image_test_features.append(image_features)
print(f"Caractéristiques des images de test extraites : {len(image_test_features)} images.")

# 8. Chargement du modèle BERT pour l'extraction de caractéristiques textuelles
print("Chargement du modèle BERT...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()
print("Modèle BERT chargé avec succès !")

# 9. Fonction pour extraire les caractéristiques textuelles avec BERT
def extract_text_features(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# 10. Extraction des caractéristiques des textes
print("Extraction des caractéristiques des textes...")
text_train_features = [extract_text_features(text) for text in df_train['text_column']]
text_test_features = [extract_text_features(text) for text in df_test['text_column']]
print(f"Caractéristiques des textes extraites : {len(text_train_features)} textes.")

# 11. Fusion des caractéristiques images et textes
print("Fusion des caractéristiques d'images et de texte...")
image_train_features = np.array(image_train_features)
image_test_features = np.array(image_test_features)
text_train_features = np.array(text_train_features)
text_test_features = np.array(text_test_features)

X_train_features = np.concatenate([image_train_features, text_train_features], axis=1)
X_test_features = np.concatenate([image_test_features, text_test_features], axis=1)
print("Fusion terminée.")

# 12. Entraînement du modèle RandomForest
print("Entraînement du modèle RandomForest...")
model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train_features, df_train['target_column'])

# 13. Évaluation du modèle
accuracy = model_rf.score(X_test_features, df_test['target_column'])
print(f"Précision du modèle : {accuracy * 100:.2f}%")


Chargement des données...
Données chargées avec succès !
Traitement des valeurs manquantes...
Valeurs manquantes traitées !
Fusion des datasets...
Fusion terminée !
Chargement du modèle BLIP...


C:\Users\a740407\AppData\Local\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
`BlipModel` is going to be deprecated in future release, please use `BlipForConditionalGeneration`, `BlipForQuestionAnswering` or `BlipForImageTextRetrieval` depending on your usecase.
Some weights of BlipModel were not initialized from the model checkpoint at Salesforce/blip-image-captioning-base and are newly initialized: ['logit_scale', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.word_embeddings.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text

Modèle BLIP chargé avec succès !
Extraction des caractéristiques des images d'entraînement...


AttributeError: 'BlipModel' object has no attribute 'visual_encoder'

In [5]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ✅ Chargement des données
print("Chargement des données...")
df_Xtrain = pd.read_csv("Data/X_train_update.csv")
df_Xtest = pd.read_csv("Data/X_test_update.csv")
df_Ytrain = pd.read_csv("Data/Y_train.csv")
df_Ytest = pd.read_csv("Data/Y_test.csv")
print("Données chargées avec succès !")

# ✅ Traitement des valeurs manquantes
print("Traitement des valeurs manquantes...")
df_Xtrain['description'] = df_Xtrain['description'].fillna("aucune description")
df_Xtest['description'] = df_Xtest['description'].fillna("aucune description")

# ✅ Fusion des features avec les labels
print("Fusion des datasets...")
df_train = df_Xtrain.merge(df_Ytrain, on="Unnamed: 0").drop(columns=["Unnamed: 0"])
df_test = df_Xtest.merge(df_Ytest, on="Unnamed: 0").drop(columns=["Unnamed: 0"])

# ✅ Extraction des features textuelles
print("Vectorisation des descriptions textuelles...")
vectorizer = TfidfVectorizer(max_features=10000)
X_train_text = vectorizer.fit_transform(df_train["description"]).toarray()
X_test_text = vectorizer.transform(df_test["description"]).toarray()

# ✅ Chargement du modèle de feature extraction d'images
print("Chargement du modèle CNN pour l'extraction des features...")
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

# ✅ Fonction d'extraction des features d'une image
def extract_image_features(image_path):
    try:
        img = image.load_img(image_path, target_size=(224, 224))  
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  
        img_array = preprocess_input(img_array)  
        features = model.predict(img_array)  
        return features.flatten()
    except:
        return np.zeros((1280,))  # Retourne un vecteur nul si l'image est absente ou corrompue

# ✅ Chargement des features des images
def get_image_features(df, image_folder):
    print(f"Extraction des features pour {len(df)} images...")
    image_features = []
    for img_id in df["imageid"]:
        img_path = os.path.join(image_folder, f"{img_id}.jpg")
        image_features.append(extract_image_features(img_path))
    return np.array(image_features)

# ✅ Extraction des features d'images
X_train_img = get_image_features(df_train, "Data/images/image_train")
X_test_img = get_image_features(df_test, "Data/images/image_test")

# ✅ Réduction de dimension des features d'images (optionnel)
print("Réduction de dimension des features d'images...")
scaler = StandardScaler()
pca = PCA(n_components=256)  # On réduit à 256 dimensions
X_train_img = pca.fit_transform(scaler.fit_transform(X_train_img))
X_test_img = pca.transform(scaler.transform(X_test_img))

# ✅ Fusion des features textuelles et visuelles
print("Fusion des features texte et image...")
X_train = np.hstack((X_train_text, X_train_img))
X_test = np.hstack((X_test_text, X_test_img))

# ✅ Définition du modèle de classification
print("Création du pipeline et début de l'entraînement...")
classifier = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# ✅ Entraînement du modèle
start_time = time.time()
classifier.fit(X_train, df_train["prdtypecode"])
training_time = time.time() - start_time
print(f"Modèle entraîné en {training_time:.2f} secondes.")

# ✅ Prédictions et évaluation
print("Prédiction et évaluation du modèle...")
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(df_test["prdtypecode"], y_pred)
print(f"Précision du modèle : {accuracy:.4f}")

print("Fin du processus ! 🎯")


Chargement des données...
Données chargées avec succès !
Traitement des valeurs manquantes...
Fusion des datasets...
Vectorisation des descriptions textuelles...
Chargement du modèle CNN pour l'extraction des features...
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step 
Extraction des features pour 84916 images...
Extraction des features pour 13812 images...
Réduction de dimension des features d'images...


C:\Users\a740407\AppData\Local\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:685: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Fusion des features texte et image...
Création du pipeline et début de l'entraînement...
Modèle entraîné en 757.33 secondes.
Prédiction et évaluation du modèle...
Précision du modèle : 0.0036
Fin du processus ! 🎯
